In [8]:
""" Imports """
import re
import numpy as np
import textwrap
import json
import urllib.request, urllib.error # For creating the JSON files
import time

In [9]:
""" Initialize and load variables with data from Riot Games' Static Data API """

champions = json.load(open("champions.json"))
items = json.load(open("items.json"))
runes = json.load(open("runes.json"))
masteries = json.load(open("mastery.json"))
summoner = json.load(open("summoner.json"))

champ_list = ""
champion_list = []
champ_description = {}
champ_abilities = {}
keys = ["Q", "W", "E", "R", "P"]
item_list = []
item_description = {}
summoner_list = []
summoner_description = {}
rune_list = []
rune_description = {}
mastery_list = []
mastery_description = {}


for key, value in champions.items():
    champion_list.append(key)
    abilities = json.load(open(key + ".json"))
    champ_abilities[(key + keys[4]).lower()] = abilities[key]["passive"]["name"] + \
                                " (Passive)" + ": " + \
                                abilities[key]["passive"]["description"]
    for i in range(0,4):
        champ_abilities[(key + keys[i]).lower()] = abilities[key]["spells"][i]["name"]\
                                + " (" + keys[i] + ")" + ": " + \
                                abilities[key]["spells"][i]["description"]
    champ_list += key + ", "
    tags = value["tags"]
    tags_string = ""
    for i in range(len(tags)):
        tags_string += tags[i]
        if i < len(tags)-1:
            tags_string += "/"
    champ_description[key.lower()] = "> " + value["name"] + ", " + value["title"] + \
                        ", is a(n) " + tags_string + ". Here's a " + \
                        "bit of lore: "+ value["blurb"] + " That's all I got." + \
                        r"¯\_(ツ)_/¯" + " You can find out more about " + value["name"]\
                        + " on the League website!"
champ_list += "and more to come!"

for key, value in items.items():
    item_list.append(items[key]["name"].lower())
    item_description[items[key]["name"].lower()] = "> " + items[key]["name"] + \
                        " is an item with the description: " + items[key]["plaintext"]\
                        + ". It costs " + str(items[key]["gold"]["total"]) + \
                        " gold to fully purchase. Google it for more information about"\
                        + " the item's stats and unique passives!"
    
for key, value in runes.items():
    rune_list.append(runes[key]["name"].lower())
    rune_description[runes[key]["name"].lower()] = "> " + runes[key]["name"] + \
                        " is a tier " + runes[key]["rune"]["tier"] + " " + \
                        runes[key]["rune"]["type"] + " rune that gives " + \
                        runes[key]["description"] + "."
    
for key, value in masteries.items():
    mastery_list.append(masteries[key]["name"].lower())
    mastery_description[masteries[key]["name"].lower()] = "> " + masteries[key]["name"]\
                        + " is a mastery with the description" + ": " + \
                        masteries[key]["description"][0] + "."

In [10]:
""" Define Helper Functions """

def check_champ(x):
    for champ in champion_list:
        if champ.lower() in x.lower():
            return True
    return False

def check_item(x):
    for item in item_list:
        if x.lower() in item.lower():
            return item
    return ""

def check_rune(x):
    for rune in rune_list:
        if x.lower() in rune.lower():
            return rune
    return ""

def check_mastery(x):
    for mast in mastery_list:
        if x.lower() in mast.lower():
            return mast
    return ""

def prnt(x):
    y = textwrap.TextWrapper(break_long_words=False,replace_whitespace=False, width=80)
    print(y.fill(x))
    
def search(x, y):
    """ Regex search, ignoring case"""
    return re.search(x, y, flags=re.I)

def match(x, y):
    """ Regex match, ignoring case"""
    return re.match(x, y, flags=re.I)

In [11]:
""" Sanity Check """

prnt(champ_abilities["annier".lower()])
prnt(item_description["rylai's crystal scepter".lower()])
prnt(rune_description["lesser mark of mana".lower()])
prnt(mastery_description["intelligence".lower()])

Summon: Tibbers (R): Annie wills her bear Tibbers to life, dealing damage to
units in the area. Tibbers can attack and also burns enemies that stand near
him.
> Rylai's Crystal Scepter is an item with the description: Abilities slow
enemies. It costs 2600 gold to fully purchase. Google it for more information
about the item's stats and unique passives!
> Lesser Mark of Mana is a tier 1 red rune that gives +3.28 mana.
> Intelligence is a mastery with the description: Your Cooldown Reduction cap is
increased to 41% and you gain 1% Cooldown Reduction.


In [12]:
def run():
    """ Runs AZILE, a chatbot that enjoys talking about League of Legends"""
    good = True
    spacer = "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" + \
            "~~~~~~~~~~~~~~~~~~~"
    beginning_statement = "> Hi I'm AZILE! I like talking with people about " + \
    "League of Legends, but I hate talking with people who forget to use " + \
    "punctuation at the end of their sentences, so don't forget! Also, it's" + \
    "hard for me to keep up, so write one sentence at a time please. Finally," +\
    " say bye to me when you gotta go! Do you understand?"
    
    prnt(beginning_statement)
    
    """ Introduction Loop """
    while True:
        print(spacer)
        arg = input() # Take in input of the user and store in variable arg
        rand = np.random.random() # Random number for Multi-response stuff
        
        if search(".*[^.!?]$", arg):
            prnt("> I told you to use punctuation!")
        elif search("[.?!](\s)+", arg):
            prnt("> I said only one sentence at a time!")
        elif search("[good]*bye[!.]$", arg) or search("[good]*bye azile", arg):
            if rand < 0.33:
                prnt("> See you later!")
            elif rand < 0.66:
                prnt("> Goodbye!")
            else: 
                prnt("> Bye!")
                print(spacer)
            return
        
        elif search("ye(s|a).*[.!?]$", arg) or search("yu(p|h).*[.!?]$", arg) \
        or search("ok(ay)*[.!?]", arg) or match("sure", arg) \
            or match("I .*(guess|suppose)", arg):
            prnt("> Great! Before we get to talking about league, introduce yourself!")
            break
        elif match("^n(o|a(w|h)).*[.!?]$", arg):
            prnt("> Well then reread my first message... Now do you understand?")            
        elif match(".*sorry.*[.!?]$", arg) or match(".*my bad.*[.!?]$", arg):
            prnt("> It's okay. Just try not to do it again. Do you understand my rules though?")    
            
        else:
            prnt("> I asked a yes or no question.")
            
    """ Main Loop"""
    asked_already1 = False
    reset1 = False
    name = ""
    
    while True:
        print(spacer)
        arg = input()  # Take in input of the user and store in variable arg
        rand = np.random.random() # Random number for Multi-response stuff
        
        if search(".*[^.!?]$", arg):
            prnt("> I told you to use punctuation! >:(")
        elif search("[.?!]+(\s)+", arg):
            prnt("> I said only one sentence at a time!")
        elif search("[good]*bye( )?[^\s]*[!.]$", arg) or \
        search("[good]*bye azile", arg) or match("see y.*", arg) \
            or search("got(ta| to) go", arg): # Check for stop interaction
            if name == "":
                name = "stranger"
            if rand < 0.33:
                prnt("> See you later " + name + "!")
                print(spacer)
            elif rand < 0.66:
                prnt("> Goodbye " + name + "!")
                print(spacer)
            else: 
                prnt("> Bye " + name + "!")
                print(spacer)
            break
            
        # League stuff
        elif search(".*(what|is) (even|what|is).*league.*$", arg)\
        or search(".*tell.*about.*league.*[.!?]$", arg) or \
        search(".*how.*play.*league.*[.!?]$", arg) \
            or search("know.*(what|how|about).*league", arg) \
        or search("give.*details", arg):
            if asked_already1:
                if search("more", arg) and not reset1:
                    print("If you want more information, look it up on Google.")
                    reset1 = True
                elif search("more", arg) and reset1:
                    print("https://na.leagueoflegends.com/en/game-info/")
                else:
                    if rand < 0.33:
                        print("> You already asked that. Scroll up!")
                    elif rand < 0.33:
                        print("> I already answered that. Scroll up!")
                    else:
                        print("> Didn't I already answer that?")
                    reset1 = True
            elif reset1:
                prnt("> Sigh... Fine here we go again.")
                prnt("> League of Legends is a 5v5 Multiplayer Online Battle Arena (MOBA) game. Players are 'summoners' who must " + \
                  "control one of the many champions available to choose each match. Summoners may customize their runes and masteries " +\
                    "to empower the champion they control. Champions battle it out on the map 'Summoner's Rift' in " + \
                 "one of the three lanes or in the jungle, gaining experience and gold by killing minions and monsters. " + \
                  "Gaining experience increases the level of a champion and gold is used to buy them items, both of which make champions strong\
                  er. Each champion has it's own unique abilities." + \
                     "The objective of the game is to destroy the other team's towers, inhibitors, and finally the enemy nexus.")
                asked_already1 = True
                reset1 = False
            else:
                prnt("> League of Legends is a 5v5 Multiplayer Online Battle Arena (MOBA) game. Players are 'summoners' who must " + \
                  "control one of the many champions available to choose each match. Summoners may customize their runes and masteries " +\
                    "to empower the champion they control. Champions battle it out on the map 'Summoner's Rift' in " + \
                 "one of the three lanes or in the jungle, gaining experience and gold by killing minions and monsters. " + \
                  "Gaining experience increases the level of a champion and gold is used to buy them items, both of which make champions strong\
                  er. Each champion has it's own unique abilities." + \
                     "The objective of the game is to destroy the other team's towers, inhibitors, and finally the enemy nexus.")
                asked_already1 = True
        elif search("(how many|what) (.+?) are (there|in)", arg) or search("what are (the )?(.+?)[ .?!]+", arg)\
        or search("list (all )?(of )?(the )?(.+?)[ .?!]+", arg):
            if search("abilit(y|ies)", arg):
                m = search("are (.+?)'(s)? abilit", arg)
                if m == None:
                    try:
                        m = search("of (.+?)[ .?!]+",arg).group(1)
                    except:
                        # do nothing
                        time.sleep(0.001)
                else:
                    try:
                        m = m.group(1)
                    except:
                        prnt("Sorry, I don't understand.")
                if check_champ(m):
                    if m[-1] == "s":
                        print("> Here are " + m + "' abilities:\n")
                    else:
                        print("> Here are " + m + "'s abilities:\n")
                    prnt(champ_abilities[(m + keys[4]).lower()])
                    print("\n")
                    for i in range(0,4):
                        prnt(champ_abilities[(m + keys[i]).lower()])
                        if i < 3:
                            print("\n")
            else:      
                m = search("(how many|what) (.+?) are (there|in)", arg)
                if m == None:
                    try:
                        m = search("what are (the )?(.+?)[ .?!]+", arg).group(2)
                    except:
                        # do nothing
                        time.sleep(0.001)
                elif m == None:
                    try:
                        m = search("list (of )?(the )?(.+?)[ .!?]+", arg).group(3)
                    except:
                        # do nothing
                        time.sleep(0.001)
                else:
                    try:
                        m = m.group(2)
                    except:
                        # do nothing
                        time.sleep(0.001)
                        m = ""
                if "champ" in m:                                                                       
                    prnt("> There are currently 134 champions in league: " + \
                         champ_list +" I can tell you more about them if you'd like.")
                elif "tower" in m or "turret" in m:
                    prnt("> There are 3 towers per team, per lane, and 2 towers guarding each team's nexus. They shoot enemy units.")
                elif "item" in m:
                    prnt("> There are hundreds of items in league! You can ask me about any of them.")
                elif "rune" in m:
                    prnt("> Runes are small, stat boosting items that can be equipped and customized outside of the rift.")
                elif "master" in m:
                    prnt("> Masteries are your special talents as a summoner. They give your champion special stats and abilities.")
                else:
                    prnt("Sorry, I don't know what you're talking about.")
        elif search("(who('s)?|what('s)?) (does )?(is )?(the )?(a(n)? )?(.+?)[ .?!]+", arg)\
        and not search("you", arg) and not search("name", arg):
            m = search("(who('s)?|what('s)?) (does )?(is )?(the )?(a(n)? )?(.+?)[ .?!]+", \
                       arg).group(9)
            if check_champ(m):
                prnt(champ_description[m.lower()])
            elif check_item(m):
                item = check_item(m)
                prnt(item_description[item])
            elif check_rune(m):
                prnt(rune_description[check_rune(m)])
            elif check_mastery(m):
                prnt(mastery_description[check_mastery(m)])
            else:
                prnt("Sorry, I don't know what you're talking about.")
        elif search("tell me.*about (.+)[.?!]", arg):
            if search("abilit(y|ies)", arg):
                m = search("about (.+?)'(s)? abilit", arg)
                if m == None:
                    try:
                        m = search("of (.+?)[ .?!]",arg).group(1)
                    except:
                        # do nothing
                        time.sleep(0.001)
                try:
                        m = m.group(1)
                except:
                        # do nothing
                        time.sleep(0.001)
                        m = ""
                if check_champ(m):
                    if m[-1] == "s":
                        print("> Here are " + m + "' abilities:\n")
                    else:
                        print("> Here are " + m + "'s abilities:\n")
                    prnt(champ_abilities[(m + keys[4]).lower()])
                    print("\n")
                    for i in range(0,4):
                        prnt(champ_abilities[(m + keys[i]).lower()])
                        if i < 3:
                            print("\n")
            else:
                m = search("tell me.*about (.+)[.?!]", arg).group(1)
                if check_champ(m):
                    prnt(champ_description[m.lower()])
                elif check_item(m):
                    item = check_item(m)
                    prnt(item_description[item])
                elif check_rune(m):
                    prnt(rune_description[m.lower()])
                elif check_mastery(m):
                    prnt(mastery_description[m.lower()])
                else:
                    prnt("Sorry, I don't know what you're talking about.")  
        # Non-League Stuff
        elif match(".*sorry.*[.!?]$", arg) or match(".*my bad.*[.!?]$", arg):
            prnt("> It's okay. Just try not to do it again. Let's talk about something else.")
        elif search("(my name(\'s| is|\?)|(I'm|I am)).*", arg) or search("what('s| is) my name", arg):
            if search("(what|do|remember)+.*((my name(\'s| is)*|(I'm|I am))[ ?.!]+$)", arg)\
            and name == "":
                prnt("> I don't think you told me your name...")
            elif name == "":
                name = search("(my name(\'s| is)|(I'm|I am)) (.+?)[ ?.!]+", arg).group(4)
                prnt("> Nice to meet you " + name + "!")
            elif search("(my name(\'s| is)|(I'm|I am)) (.+?)[ ?.!]+", arg):
                name2 = search("(my name(\'s| is)|(I'm|I am)) (.+?)[ ?.!]+", arg).group(4)
                prnt("> You told me your name was " + name + " not " + name2 +"! I'll call you "\
                     + name2 + " from now on.")
                name = name2
            else:
                prnt("> Isn't your name " + name + "?")
        elif search(".*(do |what('s| is))*you.*\?$", arg):
            if search("your name",arg):
                prnt("> My name is AZILE!")
            else:
                prnt("> I can't share that with you. You might sell my personal data to Facebook. MonkaS")
        elif match("hi( |y)?.*", arg) or match("(hello|hey).*", arg) or \
        match("what.*(s |s)*up\?", arg):
            if rand < 0.33:
                prnt("> Hi " + name + "!")
            elif rand < 0.66:
                prnt("> Hello " + name + "!")
            else:
                prnt("> Hey " + name + "!")
        elif search("ye(s|ah).*[.!?]$", arg) or search("yu(p|h).*[.!?]$", arg) or \
        search("ok(ay)*[.!?]", arg) or match("sure", arg) \
            or match("^I.*[(guess)(suppose)]+", arg):
            if rand < 0.33:
                prnt("> Okay great!")
            elif rand < 0.66:
                prnt("> Got it!")
            else:
                prnt("> I knew it!")
        elif match("(thank(s)?.)|(thank you)", arg):
            prnt("> You're welcome!")
        else:
            if rand < 0.33:
                prnt("> Sorry, I don't know what you're talking about.")
            elif rand < 0.66:
                prnt("> Huh? What are you talking about?")
            else:
                prnt("> I don't know what you mean...")

In [14]:
# EXAMPLE RUN OF AZILE
#run()

> Hi I'm AZILE! I like talking with people about League of Legends, but I hate
talking with people who forget to use punctuation at the end of their sentences,
so don't forget! Also, it'shard for me to keep up, so write one sentence at a
time please. Finally, say bye to me when you gotta go! Do you understand?
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Yup!
> Great! Before we get to talking about league, introduce yourself!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
My name is Eric.
> Nice to meet you Eric!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
What is league?
> League of Legends is a 5v5 Multiplayer Online Battle Arena (MOBA) game.
Players are 'summoners' who must control one of the many champions available to
choose each match. Summoners may customize their runes and masteries to empower
the champion they control. Champions battle it out on the map 'Summoner's Rift'
in one of th

In [ ]:
# Try yourself if you'd like! (DISCLAIMER: AZILE is not the greatest implementation so there may be bugs or bad interactions)
run()